<a href="https://colab.research.google.com/github/ptmhoang97/fiftyone_coco/blob/main/main_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
# COCO_DATASET_YEAR can be '2014' or '2017'
COCO_DATASET_YEAR = '2017'
# COCO_DATASET_TYPE can be 'train' or 'validation'
COCO_DATASET_TYPE = 'validation'
# Check below
COCO_DATASET_CLASS = 'car'

#* COCO_DATASET_CLASS
person,
bicycle,
car,
motorcycle,
airplane,
bus,
train,
truck,
boat,
traffic light,
fire hydrant,
stop sign,
parking meter,
bench,
bird,
cat,
dog,
horse,
sheep,
cow,
elephant,
bear,
zebra,
giraffe,
backpack,
umbrella,
handbag,
tie,
suitcase,
frisbee,
skis,
snowboard,
sports ball,
kite,
baseball bat,
baseball glove,
skateboard,
surfboard,
tennis racket,
bottle,
wine glass,
cup,
fork,
knife,
spoon,
bowl,
banana,
apple,
sandwich,
orange,
broccoli,
carrot,
hot dog,
pizza,
donut,
cake,
chair,
couch,
potted plant,
bed,
dining table,
toilet,
tv,
laptop,
mouse,
remote,
keyboard,
cell phone,
microwave,
oven,
toaster,
sink,
refrigerator,
book,
clock,
vase,
scissors,
teddy bear,
hair drier,
toothbrush

#1. Install FiftyOne

In [7]:
!pip install fiftyone

     |████████████████████████████████| 1.1 MB 5.3 MB/s 
     |████████████████████████████████| 631 kB 49.2 MB/s 
     |████████████████████████████████| 13.2 MB 144 kB/s 
     |████████████████████████████████| 557 kB 58.7 MB/s 
     |████████████████████████████████| 131 kB 39.1 MB/s 
     |████████████████████████████████| 29.2 MB 25 kB/s 
     |████████████████████████████████| 36.7 MB 66 kB/s 
     |████████████████████████████████| 225 kB 59.8 MB/s 
     |████████████████████████████████| 106 kB 45.3 MB/s 
     |████████████████████████████████| 79.9 MB 44 kB/s 
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 
     |████████████████████████████████| 7.9 MB 54.3 MB/s 
     |████████████████████████

In [8]:
!fiftyone --version

Subprocess ['/usr/local/lib/python3.7/dist-packages/fiftyone/db/bin/mongod', '--dbpath', '/root/.fiftyone/var/lib/mongo', '--logpath', '/root/.fiftyone/var/lib/mongo/log/mongo.log', '--port', '0', '--nounixsocket'] exited with error -11:
Uncaught exception
Traceback (most recent call last):
  File "/usr/local/bin/fiftyone", line 5, in <module>
    from fiftyone.core.cli import main
  File "/usr/local/lib/python3.7/dist-packages/fiftyone/__init__.py", line 25, in <module>
    from fiftyone.__public__ import *
  File "/usr/local/lib/python3.7/dist-packages/fiftyone/__public__.py", line 14, in <module>
    foo.establish_db_conn(config)
  File "/usr/local/lib/python3.7/dist-packages/fiftyone/core/odm/database.py", line 77, in establish_db_conn
    port = _db_service.port
  File "/usr/local/lib/python3.7/dist-packages/fiftyone/core/service.py", line 295, in port
    return self._wait_for_child_port()
  File "/usr/local/lib/python3.7/dist-packages/fiftyone/core/service.py", line 179, in _wai

In [9]:
# The version that you wish to downgrade to
#!fiftyone migrate --all -v 0.11.1
#!fiftyone migrate --info

In [10]:
import fiftyone

#2. Download images and generate annotation files

In [11]:
import fiftyone.zoo as foz

In [12]:
import os
print(os.name)

posix


In [13]:
result_name = 'coco{}_{}_{}'.format(COCO_DATASET_YEAR,COCO_DATASET_TYPE,COCO_DATASET_CLASS)
result_name_zip = '{}.zip'.format(result_name)
print(result_name)
print(result_name_zip)

coco2017_validation_car
coco2017_validation_car.zip


In [14]:
paths = {
    'COCO_DATASET_PATH': os.path.join('fiftyone', 'coco-{}'.format(COCO_DATASET_YEAR)),
    'RESULT_PATH': os.path.join('fiftyone', 'coco-{}'.format(COCO_DATASET_YEAR), result_name),
 }

In [15]:
dataset = foz.load_zoo_dataset(name = 'coco-{}'.format(COCO_DATASET_YEAR),\
                               classes = COCO_DATASET_CLASS,\
                               split = COCO_DATASET_TYPE,\
                               dataset_dir = paths['COCO_DATASET_PATH'])

 100% |██████|    1.9Gb/1.9Gb [3.1s elapsed, 0s remaining, 626.9Mb/s]      
Extracting annotations to 'fiftyone/coco-2017/raw/instances_val2017.json'
 100% |██████████████████| 535/535 [1.0m elapsed, 0s remaining, 9.1 images/s]       
Writing annotations for 535 downloaded samples to 'fiftyone/coco-2017/validation/labels.json'
Dataset info written to 'fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'validation'
 100% |█████████████████| 535/535 [6.1s elapsed, 0s remaining, 97.4 samples/s]       
Dataset 'coco-2017-validation' created


In [16]:
!mkdir {paths['RESULT_PATH']}

In [17]:
from pycocotools.coco import COCO
import os
import shutil
from tqdm import tqdm
# import skimage.io as io
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageDraw

In [18]:
resultDir=paths['RESULT_PATH']
srcImageDir = paths['COCO_DATASET_PATH'] + '/{}/{}'.format(COCO_DATASET_TYPE,'data')
srcAnnoDir = paths['COCO_DATASET_PATH'] + '/{}'.format('raw')
datasets_list=[]
if COCO_DATASET_TYPE == 'validation':
    temp = 'val' + COCO_DATASET_YEAR
    datasets_list = [temp]
elif COCO_DATASET_TYPE == 'train':
    temp = 'train' + COCO_DATASET_YEAR
    datasets_list = [temp]
classes_names = [COCO_DATASET_CLASS]
print(datasets_list)
print(classes_names)
print("resultDir: " + str(resultDir))
print("srcImageDir: " + str(srcImageDir))
print("srcAnnoDir: " + str(srcAnnoDir))

['val2017']
['car']
resultDir: fiftyone/coco-2017/coco2017_validation_car
srcImageDir: fiftyone/coco-2017/validation/data
srcAnnoDir: fiftyone/coco-2017/raw


In [19]:
headstr = """\
<annotation>
    <folder>VOC</folder>
    <filename>%s</filename>
    <source>
        <database>My Database</database>
        <annotation>COCO</annotation>
        <image>flickr</image>
        <flickrid>NULL</flickrid>
    </source>
    <owner>
        <flickrid>NULL</flickrid>
        <name>company</name>
    </owner>
    <size>
        <width>%d</width>
        <height>%d</height>
        <depth>%d</depth>
    </size>
    <segmented>0</segmented>
"""
objstr = """\
    <object>
        <name>%s</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>%d</xmin>
            <ymin>%d</ymin>
            <xmax>%d</xmax>
            <ymax>%d</ymax>
        </bndbox>
    </object>
"""
 
tailstr = '''\
</annotation>
'''
 
#if the dir is not exists,make it,else delete it
def id2name(coco):
    classes=dict()
    for cls in coco.dataset['categories']:
        classes[cls['id']]=cls['name']
    return classes
 
def write_xml(anno_path,head, objs, tail):
    f = open(anno_path, "w")
    f.write(head)
    for obj in objs:
        f.write(objstr%(obj[0],obj[1],obj[2],obj[3],obj[4]))
    f.write(tail)
 
def save_annotations_and_imgs(coco,dataset,filename,objs):
    #eg:COCO_train2014_000000196610.jpg-->COCO_train2014_000000196610.xml
    result_anno_path = '{}/{}{}'.format(resultDir,filename[:-3],'xml')
    #print(result_anno_path)
    src_img_path = '{}/{}'.format(srcImageDir,filename)
    #print(anno_path)
    #print(src_img_path)
    dst_imgpath = '{}/{}'.format(resultDir,filename)
    #print(dst_imgpath)
    img=cv2.imread(src_img_path)
    #if (img.shape[2] == 1):
    #    print(filename + " not a RGB image")
     #   return
    shutil.copy(src_img_path, dst_imgpath)
 
    head=headstr % (filename, img.shape[1], img.shape[0], img.shape[2])
    tail = tailstr
    write_xml(result_anno_path,head, objs, tail)
 
 
def showimg(coco,dataset,img,classes,cls_id,show=True):
    #global dataDir
    path = '{}/{}'.format(srcImageDir,img['file_name'])
    #print(path)
    I=Image.open(path)
    #By id, get comment information
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=cls_id, iscrowd=None)
    # print(annIds)
    anns = coco.loadAnns(annIds)
    # print(anns)
    # coco.showAnns(anns)
    objs = []
    for ann in anns:
        class_name=classes[ann['category_id']]
        if class_name in classes_names:
            #print(class_name)
            if 'bbox' in ann:
                bbox=ann['bbox']
                xmin = int(bbox[0])
                ymin = int(bbox[1])
                xmax = int(bbox[2] + bbox[0])
                ymax = int(bbox[3] + bbox[1])
                obj = [class_name, xmin, ymin, xmax, ymax]
                objs.append(obj)
                draw = ImageDraw.Draw(I)
                draw.rectangle([xmin, ymin, xmax, ymax])
    if show:
        plt.figure()
        plt.axis('off')
        plt.imshow(I)
        plt.show()
 
    return objs

for dataset in datasets_list:
    #./COCO/annotations/instances_train2014.json
    annFile='{}/instances_{}.json'.format(srcAnnoDir,dataset)
    #print(annFile)
    #COCO API for initializing annotated data
    coco = COCO(annFile)
 
    #show all classes in coco
    classes = id2name(coco)
    #print(classes)
    #[1, 2, 3, 4, 6, 8]
    classes_ids = coco.getCatIds(catNms=classes_names)
    #print(classes_ids)
    for cls in classes_names:
        #Get ID number of this class
        cls_id=coco.getCatIds(catNms=[cls])
        img_ids=coco.getImgIds(catIds=cls_id)
        #print(cls,len(img_ids))
        # imgIds=img_ids[0:10]
        for imgId in tqdm(img_ids):
            img = coco.loadImgs(imgId)[0]
            filename = img['file_name']
            # print(filename)
            objs=showimg(coco, dataset, img, classes,classes_ids,show=False)
            #print(objs)
            save_annotations_and_imgs(coco, dataset, filename, objs)

loading annotations into memory...
Done (t=0.85s)
creating index...
index created!


100%|██████████| 535/535 [00:08<00:00, 63.04it/s]


#3. Zip and copy dataset to GG drive

In [20]:
%cd {paths['COCO_DATASET_PATH']}
!ls

/content/fiftyone/coco-2017
coco2017_validation_car  info.json  raw  validation


In [23]:
!zip -r {result_name_zip} {result_name}

  adding: coco2017_validation_car/ (stored 0%)
  adding: coco2017_validation_car/000000011197.jpg (deflated 0%)
  adding: coco2017_validation_car/000000379800.xml (deflated 78%)
  adding: coco2017_validation_car/000000303908.xml (deflated 60%)
  adding: coco2017_validation_car/000000393093.xml (deflated 84%)
  adding: coco2017_validation_car/000000166664.xml (deflated 60%)
  adding: coco2017_validation_car/000000365207.xml (deflated 74%)
  adding: coco2017_validation_car/000000279714.xml (deflated 69%)
  adding: coco2017_validation_car/000000462756.xml (deflated 60%)
  adding: coco2017_validation_car/000000162858.jpg (deflated 0%)
  adding: coco2017_validation_car/000000094751.xml (deflated 69%)
  adding: coco2017_validation_car/000000184324.jpg (deflated 0%)
  adding: coco2017_validation_car/000000122166.xml (deflated 86%)
  adding: coco2017_validation_car/000000369442.jpg (deflated 0%)
  adding: coco2017_validation_car/000000464824.jpg (deflated 0%)
  adding: coco2017_validation_car/

In [24]:
!cp {result_name_zip} "/content/gdrive/MyDrive"